### import libraries

In [2]:
! pip install netCDF4

     |████████████████████████████████| 4.1MB 4.2MB/s eta 0:00:01            | 1.3MB 4.2MB/s eta 0:00:01
     |████████████████████████████████| 327kB 48.8MB/s eta 0:00:01


In [1]:
import netCDF4 # python API to work with netcdf (.nc) files
import os
import datetime
from osgeo import gdal, ogr, osr
import numpy as np # library to work with matrixes and computations in general
import matplotlib.pyplot as plt # plotting library
from auxiliary_classes import convert_time,convert_time_reverse,kelvin_to_celsius,kelvin_to_celsius_vector,Grid,Image,subImage
import json
import geojson, gdal, subprocess

### auxiliary functions

In [2]:
def print_geojson(tname, tvalue, fname, longitude, latitude, startdoc, position,endloop): #for printing to geojson - start,end,attributes
    fname = fname +".geojson"
    pmode="a"
    if startdoc==1:
        with open(fname, mode="w", encoding='utf-8') as f1: #start of geojson
            tstring = "{\n\"type\": \"FeatureCollection\",\n\"features\": ["
            print(tstring, file=f1)
            f1.close()
    else:
        if position==0: #for printing to geojson - geometry, longitude, latitude
            tstring = "\"type\": \"Feature\",\n\"geometry\": {\n\"type\": \"Point\",\n\"coordinates\": [" + str(longitude) + ","+ str(latitude) + "]\n},\n\"properties\": {"
            fname = fname 
            with open(fname, mode=pmode, encoding='utf-8') as f1:
                print(tstring, file=f1)
                f1.close()
        elif position==1:  #start of point attributes
            with open(fname, mode=pmode, encoding='utf-8') as f1:
                print("{", file=f1)
                f1.close()  
        elif position==2: #print attribute (not last)
             with open(fname, mode=pmode, encoding='utf-8') as f1:
                ttext = "\"" + str(tname) + "\": \"" +str(tvalue) + "\","
                print(ttext, file=f1) 
                f1.close() 
        elif position==3: #print last attribute
            with open(fname, mode=pmode, encoding='utf-8') as f1:
                ttext = "\"" + str(tname) + "\": \"" +str(tvalue) + "\""
                print(ttext, file=f1) 
                f1.close()        
        elif position==4: #end of point attributes
            with open(fname, mode=pmode, encoding='utf-8') as f1:  
                if endloop==0:
                    print("}\n},", file=f1)
                    f1.close()
                else:  #end of geojson
                    print("}\n}\n]\n}", file=f1)
                    f1.close()   
        

In [3]:
def probabilitydate(inputlist, probability, first): #calculate soil date with selected probability from list of soil dates of each year
    listlong = len(inputlist)
    if listlong == 1:
        outputdate = 0
        return outputdate
    elif listlong == 0:
        outputdate = 0
        return outputdate
    else:
        orderlist = orderedlist(inputlist)
        valuelist = daynumberlist(orderlist)
        value = 0
        if first==1:
            value = int(gauss_value(valuelist, probability))
        else: 
            probability=100-probability
            value = int(gauss_value(valuelist, probability))
        outputdate = orderlist[0] + timedelta(days=value)
        return outputdate

In [4]:
def trend(inputlist, nametrend,namediff, namedays,fnamesoildates): #calculate soil date with selected probability from list of soil dates of each year
    listlong = len(inputlist)
    if listlong <= 1:
        trendcoef = 0
        timediff = 0
        numberdaylist = 0
    else:
        numberdaylist = datelist2numblist(inputlist)
        x = np.arange(0,len(numberdaylist))
        y = numberdaylist
        z = np.polyfit(x,y,1)
        trendcoef=z[0]
        timediff=int(trendcoef*(listlong-1))
    print_geojson(nametrend, trendcoef, fnamesoildates, 0, 0, 0, 2, 0)
    print_geojson(namediff, timediff, fnamesoildates, 0, 0, 0, 2, 0)
    print_geojson(namedays, numberdaylist, fnamesoildates, 0, 0, 0, 2, 0)

In [5]:
def datelist2numblist(inputlist): # from list of dates to list of numbers (number of day in year)
    listlong = len(inputlist)
    outputlist = []
    firstdayinyear = date(2030, 1, 1) 
    for j in range (0,listlong,1):
        if inputlist[j]!=0:
            tempvalue = same_year(inputlist[j]) - firstdayinyear  
            outputlist.append(tempvalue.days)
      
       
    return outputlist

In [6]:
def same_year(daylong): #change date to same year (2030) for next calculation
        sdaylong = str(daylong)
        tday = int(sdaylong[8:10])
        tmonth = int(sdaylong[5:7])
        sameyear = date(2030, tmonth, tday)
   
    
        return sameyear

In [7]:
def gauss_value(inputlist, probability): #value of gaussian probability from values of input list
    
    mean = np.mean(inputlist)
    sigma = np.std(inputlist)
    values = np.random.normal(mean,sigma,10000)
    
    value = np.percentile(values,probability)
    
    return value
    

In [8]:
def orderedlist(inputlist): #sort list by date
    listlong = len(inputlist)
    for j in range (0,listlong-1,1):
        for i in range(j+1, listlong, 1):
            firstday = inputlist[j]
            secondday = inputlist[i]
            sfirstday = str(firstday)
            ssecondday = str(secondday)
            fday = int(sfirstday[8:10])
            fmonth = int(sfirstday[5:7])
            sday = int(ssecondday[8:10])
            smonth = int(ssecondday[5:7])
            if fday<10:
                firstval=str(fmonth)+"0"+str(fday)
            else:
                firstval=str(fmonth)+str(fday)
            if sday<10:
                secondval=str(smonth)+"0"+str(sday)
            else:
                secondval=str(smonth)+str(sday)
            firstvalue = int(firstval)
            secondvalue = int(secondval)
            if secondvalue < firstvalue:
                inputlist[j]=secondday
                inputlist[i]=firstday
       
    return inputlist

In [9]:
def daynumberlist(orderlist): #from ordered dates to number of days from first date 
    listlong = len(orderlist)
    outputlist=[]
    outputlist.append(0)
    for i in range(1, listlong, 1):
        difference = orderlist[i] - orderlist[0]
        outputlist.append(difference.days)
    return outputlist    
    
    

###  Find fertilizing date: function for one place

In [10]:
from datetime import date, timedelta
def findfertdate(latitude,longitude,year,soildegree,dayinrow,starthourday,endhourday,fnamesoil,im,lastlist,soilparameter):
      
       
        
    #determination of winter and summer:
    wintermonth=1
    summermonth=7
    if latitude<0:
        wintermonth=7
        summermonth=1
    
    # Last day with temperature above X degree:
    startmonth=summermonth
    endmonth=wintermonth
    lastsoilday=0
    daysbefore=0
    startdate=1
    enddate=1
    if endmonth == 1:
        endmonth=12
        enddate=31
    sdate = date(year, startmonth, startdate)   # start date for searching last soil date
    edate = date(year, endmonth, enddate)   # end date for searching last soil date
    delta = edate - sdate       # as timedelta
    for i in range(delta.days):
        daylong = sdate + timedelta(days=i)
        sdaylong = str(daylong)
        tday = int(sdaylong[8:10])
        tmonth = int(sdaylong[5:7])
        tyear = int(sdaylong[0:4])
        dayavg = [] # list for hour data
        for hour in range(starthourday, endhourday+1, 1): # for specific hours (all day,only sunrise hours,..)
            time=convert_time_reverse(datetime.datetime(tyear, tmonth, tday, hour, 0))
            slice_dictionary={'lon':[longitude,],'lat':[latitude],'time':[int(time)]}
            currenttemp=kelvin_to_celsius_vector(im.slice(soilparameter,slice_dictionary))  
            dayavg.append(currenttemp)
        dayaverage = sum(dayavg)/len(dayavg)
        if dayaverage >= soildegree:  # under soildegree?
            lastsoilday=daylong
            if daysbefore>=dayinrow-1:
                lastsoilday=daylong
            daysbefore=+1
        else:
            daysbefore=0
    
    tvarname = "LastD"+ str(year)
    print_geojson(tvarname, lastsoilday, fnamesoil, longitude, latitude, 0, 2, 0)
    if lastsoilday!= 0:
        tvalue = same_year(lastsoilday)
        lastlist.append(tvalue)
    
    
                     
    
        
                  

### Find fertilizing date: iteration by selected years

In [11]:
def fertdateyearly(latorder,lonorder,startyear,endyear,soildegree,dayinrow,starthourday,endhourday,fnamesoil,endloop,datafolder,probability,soilparameter):
    print_geojson("", "", fnamesoil, 0, 0, 0, 1,0)
    lastlist = []
       
    for year in range(startyear, endyear+1, 1):
        source = datafolder + '/' + str(year) + '.nc' 
        im=Image(netCDF4.Dataset(source,'r'))   
        longlist = im.get_data().variables['lon'][:]
        latlist= im.get_data().variables['lat'][:]
        longitude = longlist [lonorder]   
        latitude = latlist[latorder]
        if year == startyear:
            print_geojson("", "", fnamesoil, longitude, latitude, 0, 0,0)
        findfertdate(latitude,longitude,year,soildegree,dayinrow,starthourday,endhourday,fnamesoil,im,lastlist,soilparameter)
    
    nametrend = "LastDTrCo"
    namediff = "LastDdiff"
    namedays = "LastDlist"
    trend(lastlist, nametrend,namediff, namedays,fnamesoil)
    
    lastprobday = probabilitydate(lastlist, probability, 0)
    namelastprob = "LastD" + str(probability) 
    print_geojson(namelastprob, lastprobday, fnamesoil, 0, 0, 0, 3, 0)
    print_geojson("", "", fnamesoil, 0, 0, 0, 4,endloop)
          

### Find fertilizing dates: iteration by selected latitudes, longitudes

In [12]:
def fertdateplaces(startlat, startlon, endlat, endlon, startyear,endyear,soildegree,dayinrow,starthourday,endhourday,exportfolder,datafolder,fnamesoil1,probability,soilparameter,alllatlonfile):
        fnamesoil = exportfolder + "/" +fnamesoil1
        print_geojson("", "", fnamesoil, 0, 0, 1, 0,0)
        endloop=0
        
        if alllatlonfile==1:  # if it is calculated for all latitudes and longitudes in input file
            source = datafolder + '/' + str(startyear) + '.nc' 
            im=Image(netCDF4.Dataset(source,'r')) 
            arraylon = im.get_data().variables['lon'][0::]
            arraylat = im.get_data().variables['lat'][0::]
            startlat=0
            startlon=0
            endlon= len(arraylon)-1
            endlat= len(arraylat)-1
            
        for latorder in range(startlat, endlat+1, 1):
            for lonorder in range(startlon, endlon+1, 1):
                if latorder==endlat and lonorder==endlon:
                    endloop=1
                fertdateyearly(latorder,lonorder,startyear,endyear,soildegree,dayinrow,starthourday,endhourday,fnamesoil,endloop,datafolder,probability,soilparameter)
        
       
        

## <font color=red>Find fertilizing dates: input parameters and launch</font> 

In [ ]:
#Last date with soil temperature above X degree - definition:
soildegree=16 #soil temperature for fertilization # we can find last day with soil temperature above 10 / 16 .... degrees of Celsius # integer or double
dayinrow=1 #how many days in a row we consider as a last soil date with corresponding temperature #we can find last date or for example last two days of each year # integer
soilparameter="stl2" #parameter according to soil temperature depth


#Time and probability definition:
startyear=2010 #start year (integer) 
endyear=2019 #end year (integer)  
probability=50 # probability (percent) of soil date (integer 10-90)

#Optimalization:
starthourday=0 # integer 0-23
endhourday=23 # integer 0-23, >starthourday

#Files/Folders name:
datafolder = "data" #folder with data files (named by year) for each year #string
fnamesoil="fertdate" #name of created files #string
exportfolder = "export" #for all files (if each file its folder -> insert name of folder to name of file) #export folder must be created #string

#Area definition:
alllatlonfile=1 #calculate all latitudes and longitudes in input file (1=yes, 0=no)
# if alllatlonfile!=0 then:
startlat=0 # start number of list of latitudes from used netCDF4 file 
startlon=0 # start number of list of longitudes from used netCDF4 file 
endlat=48 # end number of list of latitudes from used netCDF4 file 
endlon=29 # end number of list of longitudes from used netCDF4 file 
 
fertdateplaces(startlat, startlon, endlat, endlon, startyear,endyear,soildegree,dayinrow,starthourday,endhourday,exportfolder,datafolder,fnamesoil,probability,soilparameter,alllatlonfile)






<font color=red> Output: in export folder is created geojson with points - each point has got attributes: last soil date</font> 

## From geojson to shp

In [30]:
args = ['ogr2ogr', '-f', 'ESRI Shapefile', 'export/fertdate.shp', 'export/fertdate.geojson']
subprocess.Popen(args)